In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [2]:
monthly_target_trips = pd.read_csv('monthly_target_trips.csv')
monthly_target_new_passengers = pd.read_csv('monthly_target_new_passengers.csv')
fact_trips = pd.read_csv('fact_trips.csv')
fact_passenger_summary = pd.read_csv('fact_passenger_summary.csv')
dim_repeat_trip_distribution = pd.read_csv('dim_repeat_trip_distribution.csv')
dim_date = pd.read_csv('dim_date.csv')
dim_city = pd.read_csv('dim_city.csv')
city_target_passenger_rating = pd.read_csv('city_target_passenger_rating.csv')

In [3]:
fact_trips.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8


### 1. Top and Bottom Performing Cities
### . Identify the top 3 and bottom 3 cities by total trips over the entire analysis period.

In [5]:
df1 = fact_trips.groupby(['city_id'])['trip_id'].count().reset_index(name='Trip Count')
df1.sort_values(by=['Trip Count'], ascending = [False], inplace=True)
df1 = pd.merge(df1, dim_city, on=['city_id'], how='inner')
print('Top 3 Cities')
print(df1[['city_name', 'Trip Count']].head(3))
print('-'* 50)
print('Bottom 3 Cities')
print(df1[['city_name', 'Trip Count']].tail(3))

Top 3 Cities
  city_name  Trip Count
0    Jaipur       76888
1   Lucknow       64299
2     Surat       54843
--------------------------------------------------
Bottom 3 Cities
       city_name  Trip Count
7  Visakhapatnam       28366
8     Coimbatore       21104
9         Mysore       16238


### 2. Average Fare per Trip by City
### Calculate the average fare per trip for each city and compare it with the city's average trip distance. Identify the cities with the highest and lowest average fare per trip to assess pricing efficiency across locations.

In [7]:
df2 = fact_trips.groupby(['city_id']).agg(
    Avg_Fare_Trip = ('fare_amount', 'mean'),
    Avg_Distance_Trip = ('distance_travelled(km)', 'mean')
).reset_index()
df2.sort_values(by=['Avg_Fare_Trip'], ascending=[False], inplace=True)
df2 = pd.merge(df2, dim_city, on=['city_id'], how='inner')
print(df2)
print('-'* 50)
print('Highest Average Fare Per Trip')
print(df2[['city_name', 'Avg_Fare_Trip', 'Avg_Distance_Trip']].head(1))
print('-'* 50)
print('Lowest Average Fare Per Trip')
df2[['city_name', 'Avg_Fare_Trip', 'Avg_Distance_Trip']].tail(1)

  city_id  Avg_Fare_Trip  Avg_Distance_Trip      city_name
0    RJ01     483.918128          30.023125         Jaipur
1    KL01     335.245079          24.065461          Kochi
2    CH01     283.686950          23.518714     Chandigarh
3    AP01     282.672284          22.553938  Visakhapatnam
4    KA01     249.707168          16.496921         Mysore
5    MP01     179.838609          16.502473         Indore
6    TN01     166.982183          14.979198     Coimbatore
7    UP01     147.180376          12.512963        Lucknow
8    GJ02     118.566165          11.517736       Vadodara
9    GJ01     117.272925          10.997247          Surat
--------------------------------------------------
Highest Average Fare Per Trip
  city_name  Avg_Fare_Trip  Avg_Distance_Trip
0    Jaipur     483.918128          30.023125
--------------------------------------------------
Lowest Average Fare Per Trip


,city_name,Avg_Fare_Trip,Avg_Distance_Trip
9,Surat,117.272925,10.997247


### 3. Average Ratings by City and Passenger Type
### Calculate the average passenger and driver ratings for each city, segmented by passenger type (new vs. repeat). Identify cities with the highest and lowest average ratings.


In [9]:
df3 = fact_trips.groupby(['city_id','passenger_type']).agg(
    Avg_Passenger_Rating = ('passenger_rating', 'mean'),
    Avg_Driver_Rating = ('driver_rating', 'mean')
).reset_index()
df3 = pd.merge(df3, dim_city, on=['city_id'], how='inner')
df3 = df3[['city_name', 'passenger_type', 'Avg_Passenger_Rating', 'Avg_Driver_Rating']]
df3

,city_name,passenger_type,Avg_Passenger_Rating,Avg_Driver_Rating
0,Visakhapatnam,new,8.976151,8.979995
1,Visakhapatnam,repeated,7.989628,8.992701
2,Chandigarh,new,8.489158,7.992120
3,Chandigarh,repeated,7.493798,7.472824
4,Surat,new,7.984173,6.994925
5,Surat,repeated,5.995511,6.479441
6,Vadodara,new,7.979263,7.004147
7,Vadodara,repeated,5.978629,6.481072
8,Mysore,new,8.982964,8.982878
9,Mysore,repeated,7.978495,8.965767


In [10]:
print('Highest Rated by Repated Passenger')
print(df3[df3['passenger_type']=='repeated'].sort_values(by=['Avg_Passenger_Rating'], ascending=False).head(1))
print('-'*50)
print('Lowest Rated by Repated Passenger')
print(df3[df3['passenger_type']=='repeated'].sort_values(by=['Avg_Passenger_Rating'], ascending=False).tail(1))
print('-'*50)
print('Highest Rated by New Passenger')
print(df3[df3['passenger_type']=='new'].sort_values(by=['Avg_Passenger_Rating'], ascending=False).head(1))
print('-'*50)
print('Lowest Rated by New Passenger')
print(df3[df3['passenger_type']=='new'].sort_values(by=['Avg_Passenger_Rating'], ascending=False).tail(1))

Highest Rated by Repated Passenger
   city_name passenger_type  Avg_Passenger_Rating  Avg_Driver_Rating
11     Kochi       repeated              8.003665            8.98983
--------------------------------------------------
Lowest Rated by Repated Passenger
  city_name passenger_type  Avg_Passenger_Rating  Avg_Driver_Rating
7  Vadodara       repeated              5.978629           6.481072
--------------------------------------------------
Highest Rated by New Passenger
   city_name passenger_type  Avg_Passenger_Rating  Avg_Driver_Rating
10     Kochi            new              8.987394            8.98535
--------------------------------------------------
Lowest Rated by New Passenger
   city_name passenger_type  Avg_Passenger_Rating  Avg_Driver_Rating
18   Lucknow            new              7.977429           6.990406


### 4. Peak and Low Demand Months by City
### For each city, identify the month with the highest total trips (peak demand) and the month with the lowest total trips (low demand). This analysis will help Goodcabs understand seasonal patterns and adjust resources accordingly.

In [12]:
fact_trips['date'] = pd.to_datetime(fact_trips['date'])
df4 = fact_trips.copy()
df4['month_name'] = fact_trips['date'].dt.month_name()
df4.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating,month_name
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5,January
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5,January
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8,January
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10,March
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8,June


In [13]:
df4 = df4.groupby(['city_id', 'month_name'])['trip_id'].count().reset_index(name='Trip Count')
df4['rank1'] = df4.groupby(['city_id'])['Trip Count'].rank(method = 'dense', ascending = False)
df4['rank2'] = df4.groupby(['city_id'])['Trip Count'].rank(method = 'dense', ascending = True)
df4 = df4[(df4['rank1']==1) | (df4['rank2']==1)]
df4 = pd.merge(df4, dim_city, on=['city_id'], how='inner')
df4[['city_name', 'month_name', 'Trip Count']]

,city_name,month_name,Trip Count
0,Visakhapatnam,April,4938
1,Visakhapatnam,January,4468
2,Chandigarh,April,5566
3,Chandigarh,February,7387
4,Surat,April,9831
5,Surat,January,8358
6,Vadodara,April,5941
7,Vadodara,June,4685
8,Mysore,January,2485
9,Mysore,May,3007


### 5. Weekend vs. Weekday Trip Demand by City
### Compare the total trips taken on weekdays versus weekends for each city over the six-month period. Identify cities with a strong preference for either weekend or weekday trips to understand demand variations.


In [16]:
dim_date['date'] = pd.to_datetime(dim_date['date'])

In [17]:
df5 = pd.merge(fact_trips, dim_date, on=['date'], how='inner')
df5.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating,start_of_month,month_name,day_type
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5,2024-01-01,January,Weekend
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5,2024-01-01,January,Weekday
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8,2024-01-01,January,Weekend
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10,2024-03-01,March,Weekday
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8,2024-06-01,June,Weekend


In [18]:
df5 = df5.groupby(['city_id', 'day_type'])['trip_id'].count().reset_index(name='Trip_Count')
df5

,city_id,day_type,Trip_Count
0,AP01,Weekday,15100
1,AP01,Weekend,13266
2,CH01,Weekday,19914
3,CH01,Weekend,19067
4,GJ01,Weekday,37793
5,GJ01,Weekend,17050
6,GJ02,Weekday,20310
7,GJ02,Weekend,11716
8,KA01,Weekday,6424
9,KA01,Weekend,9814


In [19]:
df5 = df5.pivot(index="city_id", columns="day_type", values="Trip_Count").reset_index()

df5.columns.name = None 
df5 = df5.rename(columns={"Weekday": "Weekday_Trips", "Weekend": "Weekend_Trips"})

df5

,city_id,Weekday_Trips,Weekend_Trips
0,AP01,15100,13266
1,CH01,19914,19067
2,GJ01,37793,17050
3,GJ02,20310,11716
4,KA01,6424,9814
5,KL01,22915,27787
6,MP01,21198,21258
7,RJ01,32491,44397
8,TN01,12576,8528
9,UP01,49617,14682


In [20]:
df5['Choice'] = np.where(df5['Weekday_Trips']>df5['Weekend_Trips'], 'People prefer traviling on Weekdays', 'People prefer traviling on Weekends')
df5 = pd.merge(df5, dim_city, on=['city_id'], how='inner')
df5[['city_name', 'Weekday_Trips', 'Weekend_Trips', 'Choice']]

,city_name,Weekday_Trips,Weekend_Trips,Choice
0,Visakhapatnam,15100,13266,People prefer traviling on Weekdays
1,Chandigarh,19914,19067,People prefer traviling on Weekdays
2,Surat,37793,17050,People prefer traviling on Weekdays
3,Vadodara,20310,11716,People prefer traviling on Weekdays
4,Mysore,6424,9814,People prefer traviling on Weekends
5,Kochi,22915,27787,People prefer traviling on Weekends
6,Indore,21198,21258,People prefer traviling on Weekends
7,Jaipur,32491,44397,People prefer traviling on Weekends
8,Coimbatore,12576,8528,People prefer traviling on Weekdays
9,Lucknow,49617,14682,People prefer traviling on Weekdays


### 6. Repeat Passenger Frequency and City Contribution Analysis
### Analyse the frequency of trips taken by repeat passengers in each city (e.g., % of repeat passengers taking 2 trips, 3 trips, etc.). Identify which cities contribute most to higher trip frequencies among repeat passengers, and examine if there are distinguishable patterns between tourism-focused and business-focused cities.


In [22]:
df6 = dim_repeat_trip_distribution.groupby(['city_id','trip_count'])['repeat_passenger_count'].sum().reset_index()
df6.head()

,city_id,trip_count,repeat_passenger_count
0,AP01,10-Trips,47
1,AP01,2-Trips,2618
2,AP01,3-Trips,1275
3,AP01,4-Trips,510
4,AP01,5-Trips,278


In [23]:
df6 = df6.pivot(index="city_id", columns="trip_count", values="repeat_passenger_count").reset_index()

# Rename columns for clarity (optional)
df6.columns.name = None  # Remove the column grouping name

df6

,city_id,10-Trips,2-Trips,3-Trips,4-Trips,5-Trips,6-Trips,7-Trips,8-Trips,9-Trips
0,AP01,47,2618,1275,510,278,163,101,71,45
1,CH01,91,1638,976,798,619,376,278,176,118
2,GJ01,117,843,1232,1430,1706,1594,1027,539,150
3,GJ02,70,429,616,718,785,829,559,251,89
4,KA01,7,720,361,188,86,60,26,21,8
5,KL01,62,3635,1857,901,494,298,161,126,92
6,MP01,109,2478,1637,967,746,494,378,235,172
7,RJ01,94,4855,2007,1173,609,400,244,184,116
8,TN01,31,286,378,397,526,450,267,157,59
9,UP01,106,927,1417,1555,1768,1937,1087,617,183


In [24]:
df6_1 = dim_repeat_trip_distribution.groupby(['city_id'])['repeat_passenger_count'].sum().reset_index()
df6_1

,city_id,repeat_passenger_count
0,AP01,5108
1,CH01,5070
2,GJ01,8638
3,GJ02,4346
4,KA01,1477
5,KL01,7626
6,MP01,7216
7,RJ01,9682
8,TN01,2551
9,UP01,9597


In [25]:
df6 = pd.merge(df6, df6_1, on=['city_id'], how='inner')
df6

,city_id,10-Trips,2-Trips,3-Trips,4-Trips,5-Trips,6-Trips,7-Trips,8-Trips,9-Trips,repeat_passenger_count
0,AP01,47,2618,1275,510,278,163,101,71,45,5108
1,CH01,91,1638,976,798,619,376,278,176,118,5070
2,GJ01,117,843,1232,1430,1706,1594,1027,539,150,8638
3,GJ02,70,429,616,718,785,829,559,251,89,4346
4,KA01,7,720,361,188,86,60,26,21,8,1477
5,KL01,62,3635,1857,901,494,298,161,126,92,7626
6,MP01,109,2478,1637,967,746,494,378,235,172,7216
7,RJ01,94,4855,2007,1173,609,400,244,184,116,9682
8,TN01,31,286,378,397,526,450,267,157,59,2551
9,UP01,106,927,1417,1555,1768,1937,1087,617,183,9597


In [26]:
trip_columns = [col for col in df6.columns if col not in ["city_id", "repeat_passenger_count"]]
for col in trip_columns:
    df6[col] = round((df6[col] / df6["repeat_passenger_count"]) * 100,2)

df6 = df6[["city_id","2-Trips", "3-Trips", "4-Trips", "5-Trips", "6-Trips", "7-Trips", "8-Trips", "9-Trips", "10-Trips"]]
df6 = pd.merge(df6, dim_city, on=['city_id'], how='inner')
df6[["city_name","2-Trips", "3-Trips", "4-Trips", "5-Trips", "6-Trips", "7-Trips", "8-Trips", "9-Trips", "10-Trips"]]

,city_name,2-Trips,3-Trips,4-Trips,5-Trips,6-Trips,7-Trips,8-Trips,9-Trips,10-Trips
0,Visakhapatnam,51.25,24.96,9.98,5.44,3.19,1.98,1.39,0.88,0.92
1,Chandigarh,32.31,19.25,15.74,12.21,7.42,5.48,3.47,2.33,1.79
2,Surat,9.76,14.26,16.55,19.75,18.45,11.89,6.24,1.74,1.35
3,Vadodara,9.87,14.17,16.52,18.06,19.08,12.86,5.78,2.05,1.61
4,Mysore,48.75,24.44,12.73,5.82,4.06,1.76,1.42,0.54,0.47
5,Kochi,47.67,24.35,11.81,6.48,3.91,2.11,1.65,1.21,0.81
6,Indore,34.34,22.69,13.40,10.34,6.85,5.24,3.26,2.38,1.51
7,Jaipur,50.14,20.73,12.12,6.29,4.13,2.52,1.90,1.20,0.97
8,Coimbatore,11.21,14.82,15.56,20.62,17.64,10.47,6.15,2.31,1.22
9,Lucknow,9.66,14.77,16.20,18.42,20.18,11.33,6.43,1.91,1.10


### 7. Monthly Target Achievement Analysis for Key Metrics 
### For each city, evaluate monthly performance against targets for total trips, new passengers, and average passenger ratings from targets_db. Determine if each metric met, exceeded, or missed the target, and calculate the percentage difference. Identify any consistent patterns in target achievement, particularly across tourism versus business-focused cities.



In [28]:
df7 = fact_trips.copy()
df7['month_name'] = df7['date'].dt.month_name()
df7 = df7.groupby(['city_id', 'month_name']).agg(
    Total_Trips = ('trip_id', 'count'),
    Avg_Passenger_Rating = ('passenger_rating', 'mean')
).reset_index()
df7.head()

,city_id,month_name,Total_Trips,Avg_Passenger_Rating
0,AP01,April,4938,8.370595
1,AP01,February,4793,8.465262
2,AP01,January,4468,8.553939
3,AP01,June,4478,8.402188
4,AP01,March,4877,8.426287


In [29]:
fact_passenger_summary['month'] = pd.to_datetime(fact_passenger_summary['month'])

In [30]:
df7_1 = fact_passenger_summary.copy()
df7_1['month_name'] = df7_1['month'].dt.month_name()
df7_1.head()

,month,city_id,new_passengers,repeat_passengers,total_passengers,month_name
0,2024-01-01,AP01,2513,650,3163,January
1,2024-01-01,CH01,3920,720,4640,January
2,2024-01-01,GJ01,2432,1184,3616,January
3,2024-01-01,GJ02,2089,544,2633,January
4,2024-01-01,KA01,1957,172,2129,January


In [31]:
df7_1 = df7_1.groupby(['city_id', 'month_name'])['new_passengers'].sum().reset_index(name='Total_New_Passengers')
df7_1.head()

,city_id,month_name,Total_New_Passengers
0,AP01,April,1845
1,AP01,February,2380
2,AP01,January,2513
3,AP01,June,1900
4,AP01,March,2170


In [32]:
df7 = pd.merge(df7, df7_1, on=['city_id', 'month_name'], how='inner')
df7.head()

,city_id,month_name,Total_Trips,Avg_Passenger_Rating,Total_New_Passengers
0,AP01,April,4938,8.370595,1845
1,AP01,February,4793,8.465262,2380
2,AP01,January,4468,8.553939,2513
3,AP01,June,4478,8.402188,1900
4,AP01,March,4877,8.426287,2170


In [33]:
monthly_target_trips['month'] = pd.to_datetime(monthly_target_trips['month'])
monthly_target_trips['month_name'] = monthly_target_trips['month'].dt.month_name()
monthly_target_trips.head()

,month,city_id,total_target_trips,month_name
0,2024-03-01,MP01,7000,March
1,2024-05-01,KA01,2500,May
2,2024-04-01,UP01,11000,April
3,2024-02-01,GJ02,6000,February
4,2024-05-01,KL01,9000,May


In [34]:
df7 = pd.merge(df7, monthly_target_trips, on=['city_id', 'month_name'], how='inner')
df7 = df7[['city_id','month_name', 'Total_Trips', 'Avg_Passenger_Rating', 'Total_New_Passengers', 'total_target_trips']]
df7.head()

,city_id,month_name,Total_Trips,Avg_Passenger_Rating,Total_New_Passengers,total_target_trips
0,AP01,April,4938,8.370595,1845,5000
1,AP01,February,4793,8.465262,2380,4500
2,AP01,January,4468,8.553939,2513,4500
3,AP01,June,4478,8.402188,1900,5000
4,AP01,March,4877,8.426287,2170,4500


In [35]:
monthly_target_new_passengers['month'] = pd.to_datetime(monthly_target_new_passengers['month'])
monthly_target_new_passengers['month_name'] = monthly_target_new_passengers['month'].dt.month_name()
monthly_target_new_passengers.head()

,month,city_id,target_new_passengers,month_name
0,2024-05-01,GJ01,1500,May
1,2024-05-01,GJ02,1500,May
2,2024-03-01,GJ01,2000,March
3,2024-05-01,UP01,2000,May
4,2024-05-01,MP01,2000,May


In [36]:
df7 = pd.merge(df7, monthly_target_new_passengers, on=['city_id', 'month_name'], how='inner')
df7 = df7.drop(['month'], axis=1)
df7.head()

,city_id,month_name,Total_Trips,Avg_Passenger_Rating,Total_New_Passengers,total_target_trips,target_new_passengers
0,AP01,April,4938,8.370595,1845,5000,2000
1,AP01,February,4793,8.465262,2380,4500,2500
2,AP01,January,4468,8.553939,2513,4500,2500
3,AP01,June,4478,8.402188,1900,5000,2000
4,AP01,March,4877,8.426287,2170,4500,2500


In [37]:
df7 = pd.merge(df7, city_target_passenger_rating, on=['city_id'], how='inner')
df7.head()

,city_id,month_name,Total_Trips,Avg_Passenger_Rating,Total_New_Passengers,total_target_trips,target_new_passengers,target_avg_passenger_rating
0,AP01,April,4938,8.370595,1845,5000,2000,8.5
1,AP01,February,4793,8.465262,2380,4500,2500,8.5
2,AP01,January,4468,8.553939,2513,4500,2500,8.5
3,AP01,June,4478,8.402188,1900,5000,2000,8.5
4,AP01,March,4877,8.426287,2170,4500,2500,8.5


In [38]:
df7['Trips_Target'] = np.where(
    df7['Total_Trips'] > df7['total_target_trips'], 
    'Exceeds Target by ' + round((df7['Total_Trips'] - df7['total_target_trips']) * 100 / df7['total_target_trips'], 2).astype(str) + '%',
    np.where(
        df7['Total_Trips'] < df7['total_target_trips'], 
        'Lags Target by ' + round((df7['Total_Trips'] - df7['total_target_trips']) * 100 / df7['total_target_trips'], 2).astype(str) + '%',
        'Same'
    )
)

df7['New_Passenger_Target'] = np.where(
    df7['Total_New_Passengers'] > df7['target_new_passengers'], 
    'Exceeds Target by ' + round((df7['Total_New_Passengers'] - df7['target_new_passengers']) * 100 / df7['target_new_passengers'], 2).astype(str) + '%',
    np.where(
        df7['Total_New_Passengers'] < df7['target_new_passengers'], 
        'Lags Target by ' + round((df7['Total_New_Passengers'] - df7['target_new_passengers']) * 100 / df7['target_new_passengers'], 2).astype(str) + '%',
        'Same'
    )
)

df7['Avg_Rating_Target'] = np.where(
    df7['Avg_Passenger_Rating'] > df7['target_avg_passenger_rating'], 
    'Exceeds Target by ' + round((df7['Avg_Passenger_Rating'] - df7['target_avg_passenger_rating']) * 100 / df7['target_avg_passenger_rating'], 2).astype(str) + '%',
    np.where(
        df7['Avg_Passenger_Rating'] < df7['target_avg_passenger_rating'], 
        'Lags Target by ' + round((df7['Avg_Passenger_Rating'] - df7['target_avg_passenger_rating']) * 100 / df7['target_avg_passenger_rating'], 2).astype(str) + '%',
        'Same'
    )
)

df7 = pd.merge(df7, dim_city, on=['city_id'], how='inner')
df7.head()

,city_id,month_name,Total_Trips,Avg_Passenger_Rating,Total_New_Passengers,total_target_trips,target_new_passengers,target_avg_passenger_rating,Trips_Target,New_Passenger_Target,Avg_Rating_Target,city_name
0,AP01,April,4938,8.370595,1845,5000,2000,8.5,Lags Target by -1.24%,Lags Target by -7.75%,Lags Target by -1.52%,Visakhapatnam
1,AP01,February,4793,8.465262,2380,4500,2500,8.5,Exceeds Target by 6.51%,Lags Target by -4.8%,Lags Target by -0.41%,Visakhapatnam
2,AP01,January,4468,8.553939,2513,4500,2500,8.5,Lags Target by -0.71%,Exceeds Target by 0.52%,Exceeds Target by 0.63%,Visakhapatnam
3,AP01,June,4478,8.402188,1900,5000,2000,8.5,Lags Target by -10.44%,Lags Target by -5.0%,Lags Target by -1.15%,Visakhapatnam
4,AP01,March,4877,8.426287,2170,4500,2500,8.5,Exceeds Target by 8.38%,Lags Target by -13.2%,Lags Target by -0.87%,Visakhapatnam


In [39]:
df7 = df7[['city_name', 'month_name', 'Total_Trips', 'total_target_trips', 'Trips_Target', 'Total_New_Passengers', 'target_new_passengers',
    'New_Passenger_Target', 'Avg_Passenger_Rating', 'target_avg_passenger_rating', 'Avg_Rating_Target']]
df7

,city_name,month_name,Total_Trips,total_target_trips,Trips_Target,Total_New_Passengers,target_new_passengers,New_Passenger_Target,Avg_Passenger_Rating,target_avg_passenger_rating,Avg_Rating_Target
0,Visakhapatnam,April,4938,5000,Lags Target by -1.24%,1845,2000,Lags Target by -7.75%,8.370595,8.50,Lags Target by -1.52%
1,Visakhapatnam,February,4793,4500,Exceeds Target by 6.51%,2380,2500,Lags Target by -4.8%,8.465262,8.50,Lags Target by -0.41%
2,Visakhapatnam,January,4468,4500,Lags Target by -0.71%,2513,2500,Exceeds Target by 0.52%,8.553939,8.50,Exceeds Target by 0.63%
3,Visakhapatnam,June,4478,5000,Lags Target by -10.44%,1900,2000,Lags Target by -5.0%,8.402188,8.50,Lags Target by -1.15%
4,Visakhapatnam,March,4877,4500,Exceeds Target by 8.38%,2170,2500,Lags Target by -13.2%,8.426287,8.50,Lags Target by -0.87%
5,Visakhapatnam,May,4812,5000,Lags Target by -3.76%,1939,2000,Lags Target by -3.05%,8.387781,8.50,Lags Target by -1.32%
6,Chandigarh,April,5566,6000,Lags Target by -7.23%,2496,3000,Lags Target by -16.8%,7.944484,8.00,Lags Target by -0.69%
7,Chandigarh,February,7387,7000,Exceeds Target by 5.53%,4104,4000,Exceeds Target by 2.6%,8.027210,8.00,Exceeds Target by 0.34%
8,Chandigarh,January,6810,7000,Lags Target by -2.71%,3920,4000,Lags Target by -2.0%,8.068135,8.00,Exceeds Target by 0.85%
9,Chandigarh,June,6029,6000,Exceeds Target by 0.48%,2430,3000,Lags Target by -19.0%,7.893681,8.00,Lags Target by -1.33%


### 8. Highest and Lowest Repeat Passenger Rate (RPR%) by City and Month 
### Analyse the Repeat Passenger Rate (RPR%) for each city across the six- month period. Identify the top 2 and bottom 2 cities based on their RPR% to determine which locations have the strongest and weakest rates.


In [41]:
df8 = fact_passenger_summary.groupby(['city_id']).agg(
    Total_Repeat_Passenger = ('repeat_passengers', 'sum'),
    Total_Passenger = ('total_passengers', 'sum')
).reset_index()
df8

,city_id,Total_Repeat_Passenger,Total_Passenger
0,AP01,5108,17855
1,CH01,5070,23978
2,GJ01,8638,20264
3,GJ02,4346,14473
4,KA01,1477,13158
5,KL01,7626,34042
6,MP01,7216,22079
7,RJ01,9682,55538
8,TN01,2551,11065
9,UP01,9597,25857


In [42]:
df8['RPR'] = round(df8['Total_Repeat_Passenger'] * 100 / df8['Total_Passenger'],2)
df8 = pd.merge(df8, dim_city, on=['city_id'], how='inner')
df8 = df8[['city_name', 'Total_Repeat_Passenger', 'Total_Passenger', 'RPR']].sort_values(by=['RPR'], ascending=[False])
print('Top 2 City in terms of RPR')
print(df8.head(2))
print('-'* 50)
print('Bottom 2 City in terms of RPR')
print(df8.tail(2))

Top 2 City in terms of RPR
  city_name  Total_Repeat_Passenger  Total_Passenger    RPR
2     Surat                    8638            20264  42.63
9   Lucknow                    9597            25857  37.12
--------------------------------------------------
Bottom 2 City in terms of RPR
  city_name  Total_Repeat_Passenger  Total_Passenger    RPR
7    Jaipur                    9682            55538  17.43
4    Mysore                    1477            13158  11.23


### Similarly, analyse the RPR% by month across all cities and identify the months with the highest and lowest repeat passenger rates. This will help to pinpoint any seasonal patterns or months with higher repeat passenger loyalty.

In [44]:
fact_passenger_summary.head()

,month,city_id,new_passengers,repeat_passengers,total_passengers
0,2024-01-01,AP01,2513,650,3163
1,2024-01-01,CH01,3920,720,4640
2,2024-01-01,GJ01,2432,1184,3616
3,2024-01-01,GJ02,2089,544,2633
4,2024-01-01,KA01,1957,172,2129


In [45]:
df8_1 = fact_passenger_summary.copy()
df8_1['month_name'] = df8_1['month'].dt.month_name()
df8_1['month_no'] = df8_1['month'].dt.month
df8_1

,month,city_id,new_passengers,repeat_passengers,total_passengers,month_name,month_no
0,2024-01-01,AP01,2513,650,3163,January,1
1,2024-01-01,CH01,3920,720,4640,January,1
2,2024-01-01,GJ01,2432,1184,3616,January,1
3,2024-01-01,GJ02,2089,544,2633,January,1
4,2024-01-01,KA01,1957,172,2129,January,1
5,2024-01-01,KL01,4865,795,5660,January,1
6,2024-01-01,MP01,2843,1033,3876,January,1
7,2024-01-01,RJ01,10423,1422,11845,January,1
8,2024-01-01,TN01,1822,392,2214,January,1
9,2024-01-01,UP01,3465,1431,4896,January,1


In [46]:
df8_1 = df8_1.groupby(['city_id', 'month_name', 'month_no']).agg(
    Total_Repeat_Passenger = ('repeat_passengers', 'sum'),
    Total_Passenger = ('total_passengers', 'sum')
).reset_index()
df8_1.head()

,city_id,month_name,month_no,Total_Repeat_Passenger,Total_Passenger
0,AP01,April,4,992,2837
1,AP01,February,2,790,3170
2,AP01,January,1,650,3163
3,AP01,June,6,802,2702
4,AP01,March,3,923,3093


In [47]:
df8_1['RPR_Month_Wise'] = round(df8_1['Total_Repeat_Passenger'] * 100 / df8_1['Total_Passenger'],2)
df8_1 = pd.merge(df8_1, dim_city, on=['city_id'], how='inner')
df8_1 = df8_1[['city_name', 'month_name', 'Total_Repeat_Passenger', 'Total_Passenger', 'RPR_Month_Wise', 'month_no']]
df8_1.sort_values(by=['city_name', 'month_no'], ascending=[True, True], inplace=True)
df8_1[['city_name', 'month_name', 'Total_Repeat_Passenger', 'Total_Passenger', 'RPR_Month_Wise']]

,city_name,month_name,Total_Repeat_Passenger,Total_Passenger,RPR_Month_Wise
8,Chandigarh,January,720,4640,15.52
7,Chandigarh,February,853,4957,17.21
10,Chandigarh,March,872,4100,21.27
6,Chandigarh,April,789,3285,24.02
11,Chandigarh,May,969,3699,26.20
9,Chandigarh,June,867,3297,26.30
50,Coimbatore,January,392,2214,17.71
49,Coimbatore,February,346,1993,17.36
52,Coimbatore,March,427,1965,21.73
48,Coimbatore,April,480,1722,27.87


In [48]:
df8_1['rnk1'] = df8_1.groupby(['city_name'])['RPR_Month_Wise'].rank(method='dense', ascending=False)
df8_1['rnk2'] = df8_1.groupby(['city_name'])['RPR_Month_Wise'].rank(method='dense', ascending=True)
df8_2 = df8_1[(df8_1['rnk1']==1) | (df8_1['rnk2']==1)]
df8_2[['city_name', 'month_name', 'Total_Repeat_Passenger', 'Total_Passenger', 'RPR_Month_Wise']]

,city_name,month_name,Total_Repeat_Passenger,Total_Passenger,RPR_Month_Wise
8,Chandigarh,January,720,4640,15.52
9,Chandigarh,June,867,3297,26.30
49,Coimbatore,February,346,1993,17.36
53,Coimbatore,May,504,1543,32.66
38,Indore,January,1033,3876,26.65
41,Indore,May,1563,3591,43.53
44,Jaipur,January,1422,11845,12.01
47,Jaipur,May,1842,7174,25.68
32,Kochi,January,795,5660,14.05
35,Kochi,May,1853,6222,29.78
